<a href="https://colab.research.google.com/github/valmirf/redes_neurais_ple/blob/master/PyTorch/Introdu%C3%A7%C3%A3o_ao_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/vmacf/redes_neurais.git

Cloning into 'redes_neurais'...
fatal: could not read Username for 'https://github.com': No such device or address


##Frameworks

---
![](https://raw.githubusercontent.com/vmacf/redes_neurais/master/KERAS/FIG/frameworks.png)


#Empresas envolvidas

![](https://raw.githubusercontent.com/vmacf/redes_neurais/master/KERAS/FIG/frameworks2.png)


#PyTorch

PyTorch é um framework de Python que funciona como uma camada em cima do Tensorflow, facilitando seu uso e facilitando procedimentos no uso de redes neurais artificiais e GPU. PyTorch tem se tornado padrão em várias empresas e institutos de pesquisa.

Nesse tutorial, vamos aprender a usar o PyTorch no treinamento e utilização de uma rede neural artificial. O tutorial seguirá várias etapas, desde a leitura de uma base de dados, até o treinamento e avaliação da rede neural treinada. 



In [ ]:
#Import packages

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
import numpy as np
import pickle
import os
from PIL import Image
import random
import time
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
import time


import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import numpy as np

#Test if GPU Cuda is available
cuda_available = torch.cuda.is_available()
print(cuda_available)

###Nesse exercício, nós treinaremos uma rede MLP para classificar imagens da base MNIST.



# 1. Carregar uma base de dados no PyTorch

A biblioteca torchvision.datasets possui as principais bases de dados disponíveis para download no PyTorch (ver: https://pytorch.org/docs/stable/torchvision/datasets.html)

Algumas bases de dados:

*   MNIST
*   COCO
*   ImageNet
*   CIFAR
*   Flickr
*   VOC
*   Cityscapes



Iremos usar a função DataLoader importada de TorchVision para carregar a base de dados MNIST. Usaremos um lote (batch) de tamanho 64 para treinamento e tamanho 1000 para teste neste conjunto de dados. O módulo TorchVision oferece ainda, muitas transformações úteis, como corte ou normalização. Os valores 0.1307 e 0.3081 são usados para a função Normalize(). Esses valores correspondem a média global e o desvio padrão do conjunto de dados MNIST. 

Usando o DataLoader, também poderíamos usar num_workers> 1, para usar subprocessos para carregar dados de forma assíncrona ou usar uma parte da memória RAM fixa (via pin_memory), para acelerar as transferências de RAM para GPU. 

In [ ]:
batch_size=64
cuda=0
train_size=50000
val_size=10000
test_size=10000
test_batch_size=1000


#trainset = datasets.MNIST('PATH_TO_STORE_TRAINSET', download=True, train=True, transform=transform)
#valset = datasets.MNIST('PATH_TO_STORE_TESTSET', download=True, train=False, transform=transform)
#trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
#valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

#normalize with mean and standard deviation
normalize = transforms.Normalize(mean=(0.1307,),std=(0.3081,))
transform = transforms.Compose([transforms.ToTensor(), normalize])

# define three datasets in order to have different transforms
# on training, validation and test
dataset_train = datasets.MNIST('/data/mnist/train', download=True, train=True, transform=transform)
#dataset_val = datasets.MNIST('/data/mnist/val', download=True, train=True, transform=transform)
dataset_test = datasets.MNIST('/data/mnist/test', download=True, train=False, transform=transform)

 
trainloader = torch.utils.data.DataLoader(dataset_train, batch_size, shuffle=True)
#valloader = torch.utils.data.DataLoader(dataset_val, batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset_test, test_batch_size)

Agora, vamos dar uma olhada em alguns exemplos da base de dados de teste:

In [ ]:
examples = enumerate(testloader)
batch_idx, (example_data, example_targets) = next(examples)

Vamos ver em que consiste um lote de dados de teste. Portanto, um lote de dados do conjunto de teste é um tensor de forma:

In [ ]:
example_data.shape

Isso significa que temos 1000 exemplos de 28x28 pixels em escala de cinza (ou seja, sem canais RGB, portanto, um). Podemos plotar alguns deles usando o matplotlib.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
fig

Ver a imagem em mais detalhes:

In [ ]:
def visualize_input(img, ax): 
    #img = np.uint8(img)
    #print('img: ', img)
    max = img.max()
    min = img.min()
    data = (img-min) / (max-min) # normalize the data to 0 - 1
    data = 255 * data # Now scale by 255
    img = np.uint8(data)
    #img = np.uint8(img.mul(255).numpy())
    #print('img: ', np.uint8(img.numpy()))
    ax.imshow(img, cmap='gray')
    width, height = img.shape
    thresh = img.max()/2.5
    for x in range(width):
        for y in range(height):
            #ax.annotate(str(round(img[x][y],2)), xy=(y,x),
            ax.annotate(str(img[x][y]), xy=(y,x),
                        horizontalalignment='center',
                        verticalalignment='center',
                        color='white' if img[x][y]<thresh else 'black')

fig = plt.figure(figsize = (12,12)) 
ax = fig.add_subplot(111)
visualize_input(example_data[0][0], ax)

Codificar os rótulos usando o One-Hot Encoding, caso precise para usar uma função como a softmax na rede. Aqui, o exemplo é ilustrativo, não a usaremos na nossa rede neural. 

In [ ]:
from sklearn.preprocessing import LabelBinarizer

# print first ten (integer-valued) training labels
#print('Integer-valued labels:')
#print(example_data[0][:10])

# each image in the MNIST dataset is represented as a 28x28x1
# image, but in order to apply a standard neural network we must
# first "flatten" the image to be simple list of 28x28=784 pixels
# Flatten MNIST images into a 784 long vector
images, labels = next(iter(trainloader))
images2, labels2 = next(iter(testloader))

print('Integer-valued labels:')
print(labels[:10])

#transform labels in one-hot condification
lb = LabelBinarizer()
labels_train_One_Hot = lb.fit_transform(labels)
labels_test_One_Hot = lb.transform(labels2)

# print first ten (one-hot) training labels
print('One-hot labels:')
print(labels_train_One_Hot[:10])

# 2. Definir o Modelo de Arquitetura

Nessa etapa, vamos definir o modelo da arquitetura da rede neural. O modelo criado terá uma camada de entrada, uma camada de saída de dez neurônios e duas camadas ocultas entre elas, uma com 128 e outra com 64 neurônios.

O módulo torch.nn permite constuir a arquitetura da rede de forma bastante simples. 

O módulo nn.Sequential envolve as camadas na rede. Existem três camadas lineares com a ativação da ReLU (uma função simples que permite a passagem de valores positivos, enquanto valores negativos são modificados para zero). A camada de saída é uma camada linear com a ativação da função LogSoftmax. Tecnicamente, uma função LogSoftmax é o logaritmo de uma função Softmax.

Além disso, temos 784 unidades na primeira camada, porque redimensionamos cada imagem antes de enviá-la para dentro da rede neural. (28 x 28 = 784)

In [ ]:
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))
print(model)

# 3. Modelo de otimização

Em seguida, definimos a função de perda como a probabilidade de log negativa. Juntas, as funções LogSoftmax() e o NLLLoss(), atuam como a perda de entropia cruzada com SoftMax.


## 3.1 Define o otimizador (SGD)

[https://pytorch.org/optimizer](https://pytorch.org/cppdocs/api/classtorch_1_1optim_1_1_optimizer.html)

### Tipos de otimizadores:
*   SGD
*   RMSprop
*   Adagrad
*   Adam


## 3.2 Especifica função de perda (categorical_crossentropy)

[https://pytorch.org/nn/](https://pytorch.org/docs/stable/nn.html#)

### Tipos de funções de perda
*   mean_squared_error 
*   mean_absolute_error
*   mean_absolute_percentage_error
*   mean_squared_logarithmic_error
*   squared_hinge
*   hinge
*   categorical_hinge
*   logcosh
*   categorical_crossentropy
*   sparse_categorical_crossentropy
*   binary_crossentropy
*   kullback_leibler_divergence
*   poisson
*   cosine_proximity


## 3.3 Como escolher uma função de perda?

### Funções de perda para regressão
*   Mean Squared Error
*   Mean Squared Logarithmic Error
*   Mean Absolute Error

### Funções de perda para classificação binária
*   Binary Cross-Entropy
*   Hinge
*   Squared Hinge 

### Funções de perda para classificação multiclasse
*   Multi-Class Cross-Entropy
*   Sparse Multiclass Cross-Entropy
*   Categorical Hinge
*   Kullback Leibler Divergence


In [ ]:
# Loss and Optimizer 
# Softmax is internally computed. 
# Set parameters to be updated. 
criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

logps = model(images) #log probabilities
loss = criterion(logps, labels) #calculate the NLL loss

#optimize funcion
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## 3.4 Atualização dos pesos

Antes do passo backward, os pesos do modelo são definidos para none. Depois de chamar a função backward(), os pesos são atualizados.

In [ ]:
print('Before backward pass: \n', model[0].weight.grad)
loss.backward()
print('After backward pass: \n', model[0].weight.grad)

# 4. Treinar o Modelo

Nessa etapa, o treinamento da rede neural é realizado. Sua rede neural repete o conjunto de treinamento e atualiza os pesos. Utilizamos o módulo torch.optim, que é um módulo do PyTorch para otimizar o modelo. Nessa arquitetura, usamos a descida de gradiente e os pesos são atualizados por backpropagation. Assim, em cada época (número de vezes que repetimos o conjunto de treinamento), veremos uma diminuição gradual do valor da função de perda.

In [ ]:
epochs = 20
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
    
        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
        #print("\nTraining Time (in minutes) =",(time()-time0)/60)

# 5. Avaliando o modelo no conjunto de teste

Agora, o modelo treinado na etapa anterior será avaliada. 


In [ ]:
images, labels = next(iter(testloader))

img = images[0].view(1, 784)

with torch.no_grad():
  logps = model(img)

ps = torch.exp(logps)
probab = list(ps.numpy()[0])
print("Predicted Digit =", probab.index(max(probab)))
print("Digit =", labels.numpy()[0])

Ness etapa, iteramos o conjunto de teste e calculamos o número total de previsões corretas. 

In [ ]:
correct_count, all_count = 0, 0
for images,labels in testloader:
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    with torch.no_grad():
        logps = model(img)

    
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

 ## 6. Salvando o modelo

Para salvar o modelo treinado, para que não seja preciso treinar novamente quando for usar o modelo, salvamos o modelo. Quando precisarmos no futuro, podemos carregá-lo e usá-lo diretamente, sem treinamento adicional.


In [ ]:
torch.save(model, './my_mnist_model.pt') 

#TESTES
1. Modifique a quantidade de camadas do modelo treinado
2. Modifique a quantidade de neurônios na(s) camada(s) intermediárias do modelo treinado
3. Se usar o modo de otimização com base no erro do conjunto de validação, quantas épocas você utilizaria para treinar seu modelo? Qual é o erro obtido com essa quantidade de épocas?
4. Modifique o tipo de otimizador do seu modelo
5. Modifique a função de perda do seu modelo
6. Avalie combinações entre tipos de otimizador, função de perda, tipos de camadas e quantidade de épocas no treinamento. Qual foi a melhor taxa de acerto no conjunto de teste? Qual a melhor configuração de modelo que você encontrou? 